In [1]:
import importlib
import sys
import torch
import pickle
import os
from tqdm.notebook import tqdm

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation


In [2]:
# Load model
#file_path_model = '../../../training_variational_dropout/sepsis/Sepsis_full_no_grad_norm_robustness.pkl'
file_path_model = '../../../training_variational_dropout/BPIC17/BPIC_2017_setting_2.pkl'
output_dir = '../../../../../evaluation_results/robustness/BPIC17/last_event_attack/'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

file_path_original = '../../../../../encoded_data/BPIC17/BPIC_2017_all_5_val.pkl'
file_path_perturbed = '../../../../../encoded_data/BPIC17/BPIC_2017_all_5_val.pkl'
file_path_redo_activity = '../../../../../encoded_data/BPIC17/val.pkl'
file_path_redo_activity_pert = '../../../../../encoded_data/BPIC17/last_event_attack.pkl'




original_dataset = torch.load(file_path_original, weights_only=False)
perturbed_dataset = torch.load(file_path_perturbed, weights_only=False)
redo_activity_dataset = torch.load(file_path_redo_activity, weights_only=False)
redo_activity_pert_dataset = torch.load(file_path_redo_activity_pert, weights_only=False)


print(f"Original dataset loaded: {len(original_dataset)} cases")
print(f"Perturbed dataset loaded: {len(perturbed_dataset)} cases")


Data set categories:  ([('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27}), ('Action', 7, {'Created': 1, 'Deleted': 2, 'EOS': 3, 'Obtained': 4, 'Released': 5, 'statechange': 6}), ('org:resource', 150, {'EOS': 1, 'User_1': 2, 'User_10': 3, 'User_100': 4, 'User_101': 5, 'User_102': 6, 'User_103': 7, 'User_104': 8, 'User_105': 9, 'User_106': 10, 'User_107': 11, 'User_108': 12, 'User_109': 13, 'User_11': 14, 'User_110': 15, 'U

In [ ]:
# Sample 10% of observations for faster inference
import random

# Set random seed for reproducibility (optional)
random.seed(42)

# Get all keys from both datasets
all_keys_orig = list(redo_activity_dataset.keys())
all_keys_pert = list(redo_activity_pert_dataset.keys())

# Ensure both datasets have the same keys
assert set(all_keys_orig) == set(all_keys_pert), "Datasets must have matching keys"

# Calculate 10% sample size
sample_size = max(1, int(len(all_keys_orig) * 0.5))

# Randomly sample 10% of the keys
sampled_keys = random.sample(all_keys_orig, sample_size)

# Create new dictionaries with only sampled keys
redo_activity_dataset = {key: redo_activity_dataset[key] for key in sampled_keys}
redo_activity_pert_dataset = {key: redo_activity_pert_dataset[key] for key in sampled_keys}

print(f"Sampled {len(sampled_keys)} observations ({len(sampled_keys)/len(all_keys_orig)*100:.1f}%) from {len(all_keys_orig)} total observations")
print(f"Original dataset now has {len(redo_activity_dataset)} entries")
print(f"Perturbed dataset now has {len(redo_activity_pert_dataset)} entries")

Sampled 17161 observations (10.0%) from 171611 total observations
Original dataset now has 17161 entries
Perturbed dataset now has 17161 entries


In [8]:
#tMp
file_path_model = '../../../training_variational_dropout/BPIC17/BPIC_2017_setting_2.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

Data set categories:  ([('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27}), ('Action', 7, {'Created': 1, 'Deleted': 2, 'EOS': 3, 'Obtained': 4, 'Released': 5, 'statechange': 6}), ('org:resource', 150, {'EOS': 1, 'User_1': 2, 'User_10': 3, 'User_100': 4, 'User_101': 5, 'User_102': 6, 'User_103': 7, 'User_104': 8, 'User_105': 9, 'User_106': 10, 'User_107': 11, 'User_108': 12, 'User_109': 13, 'User_11': 14, 'User_110': 15, 'U

In [ ]:
# Create evaluation instances (NON-RANDOM ORDER)
eval_original = ProbabilisticEvaluation(
    model, original_dataset,
    concept_name='concept:name',
    num_processes=1, 
    growing_num_values=['case_elapsed_time'],
    samples_per_case=1000,
    sample_argmax=False,
    use_variance_cat=True,
    use_variance_num=True,
    all_cat=['concept:name', 'org:resource', 'lifecycle:transition'],
    all_num=['case_elapsed_time', 'event_elapsed_time'],
    dataset_predefined_prefixes=redo_activity_dataset
)

eval_perturbed = ProbabilisticEvaluation(
    model, perturbed_dataset,
    concept_name='concept:name', #'Activity'
    num_processes=1,
    growing_num_values=['case_elapsed_time'],
    samples_per_case=1000,
    sample_argmax=False,
    use_variance_cat=True,
    use_variance_num=True,
    all_cat=['concept:name', 'org:resource', 'lifecycle:transition'],
    all_num=['case_elapsed_time', 'event_elapsed_time'],
    dataset_predefined_prefixes=redo_activity_pert_dataset
)

print("ProbabilisticEvaluation instances created")


ProbabilisticEvaluation instances created


In [10]:
# Import robustness metrics module
import robustness.evaluator.robustness_metrics
#importlib.reload(robustness.robustness_metrics)
from robustness.evaluator.robustness_metrics import save_chunk

print("Robustness metrics module imported")

# Helper functions for filtering predictions and calculating remaining time
def filter_prediction_events(prediction_list, concept_name='concept:name'):
    """Filter prediction events to only include concept:name and case_elapsed_time"""
    if prediction_list is None:
        return None
    filtered = []
    for event in prediction_list:
        if not isinstance(event, dict):
            continue
        filtered_event = {}
        if concept_name in event:
            filtered_event[concept_name] = event[concept_name]
        if 'case_elapsed_time' in event:
            filtered_event['case_elapsed_time'] = event['case_elapsed_time']
        filtered.append(filtered_event)
    return filtered

def calculate_remaining_time(prefix, prediction, concept_name='concept:name'):
    """Calculate remaining time from prefix and prediction"""
    if not prefix or not prediction:
        return None
    if not isinstance(prefix[-1], dict) or 'case_elapsed_time' not in prefix[-1]:
        return None
    if not isinstance(prediction[-1], dict) or 'case_elapsed_time' not in prediction[-1]:
        return None
    current_time = prefix[-1]['case_elapsed_time']
    final_time = prediction[-1]['case_elapsed_time']
    return final_time - current_time

def calculate_sampled_remaining_times(prefix, predicted_suffixes, concept_name='concept:name'):
    """Calculate remaining time for each sample in predicted_suffixes"""
    if not prefix or not predicted_suffixes:
        return None
    if not isinstance(prefix[-1], dict) or 'case_elapsed_time' not in prefix[-1]:
        return None
    current_time = prefix[-1]['case_elapsed_time']
    
    remaining_times = []
    for sample in predicted_suffixes:
        if not sample or not isinstance(sample[-1], dict) or 'case_elapsed_time' not in sample[-1]:
            remaining_times.append(None)
        else:
            final_time = sample[-1]['case_elapsed_time']
            remaining_times.append(final_time - current_time)
    return remaining_times


Robustness metrics module imported


In [11]:
# Main evaluation loop
os.makedirs(output_dir, exist_ok=True)

save_every = 50
results = {}
concept_name = 'concept:name'  # Match the concept_name used in ProbabilisticEvaluation

for i, ((case_name_orig, prefix_len_orig, prefix_orig, predicted_suffixes_orig, suffix_orig, mean_pred_orig),
        (case_name_pert, prefix_len_pert, prefix_pert, predicted_suffixes_pert, suffix_pert, mean_pred_pert)) in enumerate(
    tqdm(zip(eval_original.evaluate_with_predifined_prefix(random_order=False), 
             eval_perturbed.evaluate_with_predifined_prefix(random_order=False)), 
         desc="Evaluating robustness")):
    
    # Ensure we're comparing the same case and prefix length
    assert case_name_orig == case_name_pert, f"Case mismatch: {case_name_orig} != {case_name_pert}"

    assert prefix_len_orig == prefix_len_pert, f"Prefix length mismatch: {prefix_len_orig} != {prefix_len_pert}"

    # Filter predictions to only include concept:name and case_elapsed_time
    mean_pred_orig_filtered = filter_prediction_events(mean_pred_orig, concept_name=concept_name)
    predicted_suffixes_orig_filtered = [filter_prediction_events(sample, concept_name=concept_name) 
                                        for sample in predicted_suffixes_orig] if predicted_suffixes_orig else None
    
    mean_pred_pert_filtered = filter_prediction_events(mean_pred_pert, concept_name=concept_name)
    predicted_suffixes_pert_filtered = [filter_prediction_events(sample, concept_name=concept_name) 
                                        for sample in predicted_suffixes_pert] if predicted_suffixes_pert else None
    
    # Calculate remaining times immediately
    mean_pred_remaining_time_orig = calculate_remaining_time(prefix_orig, mean_pred_orig, concept_name=concept_name)
    sampled_remaining_time_orig = calculate_sampled_remaining_times(prefix_orig, predicted_suffixes_orig, concept_name=concept_name)
    
    mean_pred_remaining_time_pert = calculate_remaining_time(prefix_pert, mean_pred_pert, concept_name=concept_name)
    sampled_remaining_time_pert = calculate_sampled_remaining_times(prefix_pert, predicted_suffixes_pert, concept_name=concept_name)

    # Store results with new structure
    key = (case_name_orig, prefix_len_orig)
    results[key] = {
        'original': (
            prefix_orig,  # Keep all fields
            suffix_orig,  # Keep all fields
            mean_pred_orig_filtered,  # Filtered: only concept:name and case_elapsed_time
            predicted_suffixes_orig_filtered,  # Filtered: only concept:name and case_elapsed_time
            mean_pred_remaining_time_orig,  # NEW: single float
            sampled_remaining_time_orig  # NEW: list of floats
        ),
        'perturbed': (
            prefix_pert,  # Keep all fields
            suffix_pert,  # Keep all fields
            mean_pred_pert_filtered,  # Filtered: only concept:name and case_elapsed_time
            predicted_suffixes_pert_filtered,  # Filtered: only concept:name and case_elapsed_time
            mean_pred_remaining_time_pert,  # NEW: single float
            sampled_remaining_time_pert  # NEW: list of floats
        ),
    }

    
    if (i + 1) % save_every == 0:
        save_chunk(results, i, output_dir)
        results = {}

if len(results):
    save_chunk(results, i, output_dir)

print("Robustness evaluation completed!")


Evaluating robustness: 0it [00:00, ?it/s]

  0%|          | 0/17161 [00:00<?, ?it/s]

  0%|          | 0/17161 [00:00<?, ?it/s]

Saved 50 results to ../../../../../evaluation_results/robustness/BPIC17/last_event_attack/robustness_results_part_050.pkl


KeyboardInterrupt: 

In [6]:
# Load all saved chunks and combine them
all_results = {}
# Get all chunk files and sort them
chunk_files = [f for f in os.listdir(output_dir) if f.startswith('robustness_results_part_')]
chunk_files.sort()  # Ensure correct order

print(f"Found {len(chunk_files)} chunk files")

for chunk_file in chunk_files:
    chunk_path = os.path.join(output_dir, chunk_file)
    print(f"Loading {chunk_file}...")
    with open(chunk_path, 'rb') as f:
        chunk_results = pickle.load(f)
        all_results.update(chunk_results)
        print(f"  Added {len(chunk_results)} results from {chunk_file}")

# Also add the final results if any (e.g. from a still-running evaluation loop)
if 'results' in locals() and len(results) > 0:
    print(f"Adding final {len(results)} results...")
    all_results.update(results)

print(f"\nTotal results loaded: {len(all_results)}")

# Save combined results into a single pickle file
combined_results_path = os.path.join(output_dir, 'robustness_results.pkl')
with open(combined_results_path, 'wb') as f:
    pickle.dump(all_results, f)

print(f"Combined results saved to {combined_results_path}")

Found 12 chunk files
Loading robustness_results_part_050.pkl...
  Added 50 results from robustness_results_part_050.pkl
Loading robustness_results_part_100.pkl...
  Added 50 results from robustness_results_part_100.pkl
Loading robustness_results_part_150.pkl...
  Added 50 results from robustness_results_part_150.pkl
Loading robustness_results_part_200.pkl...
  Added 50 results from robustness_results_part_200.pkl
Loading robustness_results_part_250.pkl...
  Added 50 results from robustness_results_part_250.pkl
Loading robustness_results_part_300.pkl...
  Added 50 results from robustness_results_part_300.pkl
Loading robustness_results_part_350.pkl...
  Added 50 results from robustness_results_part_350.pkl
Loading robustness_results_part_400.pkl...
  Added 50 results from robustness_results_part_400.pkl
Loading robustness_results_part_450.pkl...
  Added 50 results from robustness_results_part_450.pkl
Loading robustness_results_part_500.pkl...
  Added 50 results from robustness_results_pa